In [14]:
%matplotlib inline
import pandas as pd
import numpy as np
from statsmodels.api import OLS, add_constant
import pandas_datareader.data as web
from linearmodels.asset_pricing import LinearFactorModel
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
sns.set_style('whitegrid')

In [16]:
ff_factor = 'F-F_Research_Data_5_Factors_2x3'
ff_factor_data = web.DataReader(ff_factor, 'famafrench', start='2010', end='2017-12')[0]
ff_factor_data.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 96 entries, 2010-01 to 2017-12
Freq: M
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mkt-RF  96 non-null     float64
 1   SMB     96 non-null     float64
 2   HML     96 non-null     float64
 3   RMW     96 non-null     float64
 4   CMA     96 non-null     float64
 5   RF      96 non-null     float64
dtypes: float64(6)
memory usage: 5.2 KB


In [17]:
ff_factor_data.describe()

,Mkt-RF,SMB,HML,RMW,CMA,RF
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,1.158750,0.054062,-0.051771,0.126042,0.052812,0.012604
std,3.580187,2.290739,2.191621,1.591052,1.409858,0.022583
min,-7.890000,-4.510000,-4.520000,-3.930000,-3.350000,0.000000
25%,-0.917500,-1.660000,-1.627500,-1.160000,-0.965000,0.000000
50%,1.235000,0.190000,-0.305000,0.135000,-0.015000,0.000000
75%,3.197500,1.517500,1.142500,1.140000,0.927500,0.010000
max,11.350000,6.800000,8.220000,3.530000,3.780000,0.090000


In [18]:
ff_portfolio = '17_Industry_Portfolios'
ff_portfolio_data = web.DataReader(ff_portfolio, 'famafrench', start='2010', end='2017-12')[0]
ff_portfolio_data = ff_portfolio_data.sub(ff_factor_data.RF, axis=0)
ff_portfolio_data.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 96 entries, 2010-01 to 2017-12
Freq: M
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Food    96 non-null     float64
 1   Mines   96 non-null     float64
 2   Oil     96 non-null     float64
 3   Clths   96 non-null     float64
 4   Durbl   96 non-null     float64
 5   Chems   96 non-null     float64
 6   Cnsum   96 non-null     float64
 7   Cnstr   96 non-null     float64
 8   Steel   96 non-null     float64
 9   FabPr   96 non-null     float64
 10  Machn   96 non-null     float64
 11  Cars    96 non-null     float64
 12  Trans   96 non-null     float64
 13  Utils   96 non-null     float64
 14  Rtail   96 non-null     float64
 15  Finan   96 non-null     float64
 16  Other   96 non-null     float64
dtypes: float64(17)
memory usage: 13.5 KB


In [19]:

ff_portfolio_data.describe()

,Food,Mines,Oil,Clths,Durbl,Chems,Cnsum,Cnstr,Steel,FabPr,Machn,Cars,Trans,Utils,Rtail,Finan,Other
count,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000,96.000000
mean,1.045625,0.197083,0.547917,1.396979,1.155208,1.303229,1.136250,1.731354,0.555625,1.350521,1.227604,1.278854,1.465000,0.890312,1.234375,1.241562,1.282396
std,2.795857,7.902185,5.577552,5.025167,5.137482,5.594216,3.174283,5.246518,7.389824,4.694408,4.811242,5.718887,4.150833,3.235140,3.508655,4.809791,3.708972
min,-5.170000,-24.380000,-12.010000,-10.000000,-13.210000,-17.390000,-7.300000,-13.960000,-20.490000,-11.960000,-9.080000,-11.650000,-8.560000,-6.990000,-9.180000,-11.040000,-7.920000
25%,-0.785000,-5.847500,-3.167500,-1.865000,-2.017500,-1.445000,-0.920000,-2.462500,-4.410000,-1.447500,-2.047500,-1.245000,-0.880000,-0.745000,-0.962500,-1.467500,-1.075000
50%,0.930000,-0.460000,1.040000,1.160000,1.205000,1.435000,1.470000,2.190000,0.660000,1.485000,1.545000,0.645000,1.505000,1.215000,0.880000,1.955000,1.575000
75%,3.187500,5.715000,3.915000,3.857500,4.322500,4.442500,3.317500,5.390000,4.220000,3.837500,4.657500,4.802500,4.235000,2.952500,3.355000,4.092500,3.517500
max,6.670000,21.920000,16.300000,17.200000,16.580000,18.370000,8.290000,15.550000,21.350000,17.660000,14.650000,20.860000,13.160000,7.900000,12.360000,13.480000,10.790000


In [21]:
with pd.HDFStore('data/assets.h5') as store:
    prices = store['/quandl/wiki/prices'].adj_close.unstack().loc['2010':'2017']
    equities = store['/us_equities/stocks'].drop_duplicates()

KeyError: 'No object named /quandl/wiki/prices in the file'